In [ ]:
import random
import sympy as sp
import networkx as nx
import matplotlib.pyplot as plt
import torch
from torch import nn
!pip install torch-geometric
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch.optim.lr_scheduler import StepLR

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.0 MB/s eta 0:00:00


### 1. DATASET GENERATION

In [ ]:
def generate_equation_problem():
    x = sp.symbols('x')
    coeff = random.randint(1, 3)  # Reduced range for coefficients
    constant = random.randint(1, 5)  # Reduced range for constants
    rhs = random.randint(1, 20)  # Reduced range for right-hand side
    eq = sp.Eq(sp.expand(coeff * (x + constant)**2), rhs)
    solutions = sp.solve(eq, x)
    return eq, [float(s) for s in solutions if sp.im(s) == 0]  # Real solutions only

def generate_dataset(problem_function, size=2000):  # Doubled dataset size
    data = []
    for _ in range(size):
        problem, solution = problem_function()
        data.append(problem_to_pyg_data(problem, solution))
    return data

### 2. GRAPH REPRESENTATION

In [ ]:
def problem_to_graph(problem):
    graph = nx.DiGraph()
    nodes = {}

    def add_node(expr, parent=None):
        if expr in nodes:
            return nodes[expr]
        node_id = len(nodes)
        nodes[expr] = node_id
        graph.add_node(node_id, label=str(expr))
        if parent is not None:
            graph.add_edge(parent, node_id)
        if isinstance(expr, sp.Basic):
            for arg in expr.args:
                add_node(arg, node_id)
        return node_id

    add_node(problem)
    return graph

def problem_to_pyg_data(problem, solution):
    graph = problem_to_graph(problem)
    edge_index = torch.tensor(list(graph.edges())).t().contiguous() if len(graph.edges()) > 0 else torch.zeros((2, 0), dtype=torch.long)

    # Extract features: Coefficients, constants, and operators as node features
    node_features = []
    for node, data in graph.nodes(data=True):
        label = data["label"]
        if label.isdigit():
            node_features.append([float(label), 0, 0, 0])  # Coefficient, constant, power, operator
        elif "x" in label:
            node_features.append([0, 1, 1, 0])  # Variable features
        elif label in ["+", "-", "*", "/"]:
            node_features.append([0, 0, 0, 1])  # Operator features
        else:
            node_features.append([0, 0, 0, 0])  # Unknowns

    x = torch.tensor(node_features, dtype=torch.float)
    y = torch.tensor([solution[0]] if solution else [0.0], dtype=torch.float)  # First real solution

    return Data(x=x, edge_index=edge_index, y=y)

# Generate dataset
equation_data = generate_dataset(generate_equation_problem, 5000)
data_loader = DataLoader(equation_data, batch_size=64, shuffle=True)  # Increased batch size

### 3. MODEL DEFINITION

In [ ]:
class GNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, output_dim)
        self.fc1 = nn.Linear(output_dim, 64)
        self.fc2 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.conv3(x, edge_index)
        x = global_mean_pool(x, batch)
        x = self.dropout(torch.relu(self.fc1(x)))
        return self.fc2(x)

### 4. TRAINING AND EVALUATION

In [ ]:
def train_model(model, data_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for data in data_loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out.squeeze(), data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(data_loader)

def evaluate_model(model, data_loader):
    model.eval()
    total_mae = 0
    with torch.no_grad():
        for data in data_loader:
            out = model(data.x, data.edge_index, data.batch)
            mae = torch.abs(out.squeeze() - data.y).mean()
            total_mae += mae.item()
    return total_mae / len(data_loader)

### 5. MAIN EXECUTION



In [ ]:
# Initialize model and hyperparameters
input_dim = 4  # Coefficients, constants, powers, and operators
hidden_dim = 256
output_dim = 128
model = GNNModel(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.MSELoss()
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)

# Training loop
for epoch in range(50):
    loss = train_model(model, data_loader, optimizer, criterion)
    scheduler.step()
    print(f"Epoch {epoch + 1}, Loss: {loss}")

# Evaluation
mae = evaluate_model(model, data_loader)
accuracy = max(0, 100 - mae * 10)  # Accuracy out of 100
print("Mean Absolute Error:", mae)
print("Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1, Loss: 10.785987437525883
Epoch 2, Loss: 7.95008926150165
Epoch 3, Loss: 7.8197392149816585
Epoch 4, Loss: 7.937989210780663
Epoch 5, Loss: 7.776319769364369
Epoch 6, Loss: 7.847091578230073
Epoch 7, Loss: 7.5997750185712984
Epoch 8, Loss: 7.541442237322843
Epoch 9, Loss: 7.558936710599102
Epoch 10, Loss: 7.3195333842989765
Epoch 11, Loss: 7.072226880471917
Epoch 12, Loss: 6.967530829997003
Epoch 13, Loss: 6.911710684812522
Epoch 14, Loss: 6.811581327945372
Epoch 15, Loss: 6.688727855682373
Epoch 16, Loss: 6.529605156258691
Epoch 17, Loss: 6.466413494906848
Epoch 18, Loss: 6.483349517176423
Epoch 19, Loss: 6.393912303296825
Epoch 20, Loss: 6.429590158824679
Epoch 21, Loss: 6.281494243235528
Epoch 22, Loss: 6.2823139504541325
Epoch 23, Loss: 6.391000765788404
Epoch 24, Loss: 6.238706727571126
Epoch 25, Loss: 6.2879912279829195
Epoch 26, Loss: 6.118708350990392
Epoch 27, Loss: 6.233451776866671
Epoch 28, Loss: 6.18025084386898
Epoch 29, Loss: 6.1374319716344905
Epoch 30, Loss: 6.

In [ ]:
x = sp.symbols('x')
coeff = random.randint(1, 3)  # Reduced range for coefficients
constant = random.randint(1, 5)  # Reduced range for constants
rhs = random.randint(1, 20)  # Reduced range for right-hand side
eq = sp.Eq(sp.expand(coeff * (x + constant)**2), rhs)
solutions = sp.solve(eq, x)
eq, [float(s) for s in solutions if sp.im(s) == 0]

(Eq(3*x**2 + 12*x + 12, 5), [-3.290994448735806, -0.7090055512641944])

In [ ]:
def generate_equation_problem():
    x = sp.symbols('x')
    coeff = random.randint(1, 3)  # Reduced range for coefficients
    constant = random.randint(1, 5)  # Reduced range for constants
    rhs = random.randint(1, 20)  # Reduced range for right-hand side
    eq = sp.Eq(sp.expand(coeff * (x + constant)**2), rhs)
    solutions = sp.solve(eq, x)
    return eq, [float(s) for s in solutions if sp.im(s) == 0]

def visualize_graph(graph):
    # Set layout and node labels
    pos = nx.spring_layout(graph, seed=42)  # You can adjust the layout
    labels = nx.get_node_attributes(graph, 'label')

    # Draw the graph
    plt.figure(figsize=(10, 8))
    nx.draw(graph, pos, with_labels=True, labels=labels, node_size=3000, node_color='skyblue', font_size=10, font_weight='bold', edge_color='gray')
    plt.title("Graph Representation of the Equation")
    plt.show()

example_equation, _ = generate_equation_problem()
graph = problem_to_graph(example_equation)
visualize_graph(graph)